In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pycountry # match country names to iso code
import geoplot as gplt #plotting maps
import geopandas as gpd
import geoplot.crs as gcrs
import imageio #not used ?
import pathlib #not used ?
import matplotlib.pyplot as plt
import mapclassify as mc #???
import os.path
from pickling import pickle_load
from helpers import * #custom made functions
print("import completed")




 The data for the `Data` folder can be downloaded from [here](https://drive.google.com/drive/folders/1Wz6D2FM25ydFw_-41I9uTwG9uNsN4TCF)

 The initial directory structure for this notebook should be the following:

 ├── Data/
 │   ├── BeerAdvocate/
 │       ├── beers.csv
 |       ├── breweries.csv
 |       ├── users.csv
 |       ├── reviews.txt
 |       ├── ratings.txt
 │   ├── RateBeer/
 │       ├── beers.csv
 │       ├── breweries.csv
 │       ├── users.csv
 |       ├── reviews.txt
 │       ├── ratings.txt
 │   ├── matched_beer_data/
 │       ├── beers.csv
 |       ├── breweries.csv
 |       ├── ratings.csv
 |       ├── ratings_with_text_ba.txt
 |       ├── ratings_with_text_rb.txt
 |       ├── ratings_ba.txt
 |       ├── ratings_rb.txt
 |       ├── users.csv
 |       ├── users_approx.csv
 ├── README.md
 ├── notebook.ipynb
 ├── requirements.txt
 └── .gitignore
data folder path

 # Table Description
 TODO: finish these, add data format and make it pretty
 ### Beers

 Source of column description: https://www.beeradvocate.com/community/threads/beeradvocate-ratings-explained.184726/

 |Column name |Data type|
 |-----|--------|
 |beer_id|3       |
 |Egg  |12      |
 -
 - name
 - brewery_id
 - brewery_name
 - style
 - nbr_ratings
 - nbr_reviews
 - avg
 - ba_score (The BA Score is the beer's overall score based on its ranking within its style category. It's based on the beer's truncated (trimmed) mean and a custom Bayesian (weighted rank) formula that takes the beer's style into consideration. Its purpose is to provide consumers with a quick reference using a format that's familiar to the wine and liquor worlds.)
       95-100 = world-class
       90-94 = outstanding
       85-89 = very good
       80-84 = good
       70-79 = okay
       60-69 = poor
       < 60 = awful
 - bros_score
 - abv (alcohol by volume)
 - avg_computed
 - zscore
 - nbr_matched_valid_ratings
 - avg_matched_valid_ratings


 ### Breweries
 - brewery_id -> id
 - location
 - name
 - nbr_beers


 ### Users
 - nbr_ratings
 - nbr_reviews
 - user_id
 - user_name
 - joined
 - location


 ### Reviews
 The difference in columns between reviews and ratings is only the missing boolean
 - beer_name
 - beer_id
 - brewery_name
 - brewery_id
 - style
 - abv
 - date
 - user_name
 - user_id
 - appearance
 - aroma
 - palate
 - taste
 - overall
 - rating
 - text
 - review




 #Subsetting to the Matched Beer/Brewery Data
 We conduct our analyses separately for both the BeerAdvocate and RateBeer datasets in order to be able to compare our results
 and check if they are robust between the two datasets. The goal of this is to try and remove an extra source of variation
 due to the different user bases on the two websites.

 Based on the merged dataset, we choose to subset beers and breweries that are presents in both dataset. We then recover
 TODO: make prettier
 column dropping justification:
 - ba_score: only available in ba dataset, not really linear, not available for every review (only for the aggregation of all reviews) and hard to interpret (rather use user avg.)
 - bros_score: wtf ? only available in ba dataset
 - anything related to reviews
 - overall_score: same argument as ba_score
 - avg_matched_...: we don't care about matching the two datasets since we're testing each hypothesis two times

 Also dropped some rows with nans
 the rest we keep just in case

issues:
 - what is the zscore
 - what is the avg_matched_valid_ratings ?
 - what is diff and sim ?

 # Pipeline
 ### Load and filter Merged Beer/brewery Data
 #### df_beers
loading and naming columns
infos:
so we lost about 84% of the beers, but still 45k is significative.
is it interesting to compute the avg nb of reviews for beers we lost ? => don't know how to do (A \ AintersectB)
nan checks:
not sure: we remove beers that didn't get at least 1 review in each ds ?
no NAN lefts !
do abv contents are the same in both datasets? Yes they are
do names match ? Most of them don't. BA don't include the brewery name in the beer name but RB often does
we can just choose on of the two name arbitrarily (beera advocate names looks more complete)
 ### Breweries
loading and renaming columns
updating breweries list and beer count according to the beer we removed in the previous section
    try to find an element in a dataframe, returns 0 if not found.
do number of beers per brewery matches between the two dataset ?
no, but we notice that it is ony due to minor difference in the name, so we leave them as it is

 ### Create dataframes for later use

 #### Beer Advocate Ratings
 ##### Load BA User Data
 We do not know 1 username, 2652 join dates and 31279 locations (i.e. they are nan)
location processing
removing html tags
separating country from states (for USA)
 54 missing country codes...
 ##### Load BA Ratings
 TODO Explain difference between ratings and reviews
 Explain why uncommented (RAM)
 df_ba_ratings = pickle_load(path_ba + "ratings.txt",load_with_text=True)
 df_ba_ratings.set_index(df_ba_ratings["date"], inplace = True) # Set date as index for ratings
 df_ba_ratings = df_ba_ratings.convert_dtypes(infer_objects=True) # infer correct string types (does not filter nan values)
 df_ba_ratings[["overall","taste","palate","aroma","appearance","abv"]] = df_ba_ratings[["overall","taste","palate","aroma","appearance","abv"]].apply(lambda x : pd.to_numeric(x,errors="coerce")) # coerce to numeric
 df_ba_ratings = df_ba_ratings.astype({"review":bool}) # convert to boolean
 df_ba_ratings["text"] = df_ba_ratings["text"].apply(lambda x: x if str.lower(x) != "nan" else "") # convert nan to empty string
 df_ba_ratings = df_ba_ratings.dropna() # drop nan values
 print(df_ba_ratings.info()) # print info
 print("Number of nan values",df_ba_ratings.isna().sum())
 print("Number of non-reviews:",df_ba_ratings[df_ba_ratings["review"] == False].shape[0])
 print("Number of text that is 'nan':",df_ba_ratings[df_ba_ratings["text"].str.lower() == "nan"].shape[0])
 for column in ["beer_name","beer_id","brewery_name","brewery_name","style","user_name","user_id"]:
         print(f"number of values in {column} that is 'nan'", df_ba_ratings[df_ba_ratings[column].str.lower() == "nan"].shape[0])
 TODO: convert to style to categorical, it might make sense to also change datatype of beer_id and brewery_id to numeric
 ##### Merge BA Ratings and Users for beers from the matched Dataset
 Filter BA Ratings to only keep beers in matched beer dataset
 df_md_beer_ids = df_beers["beer_id_ba"].unique().astype(str)
 df_ba_beer_ids = df_ba_ratings["beer_id"].unique()
 df_ba_ratings_filtered_md_beers = df_ba_ratings[df_ba_ratings["beer_id"].isin(df_md_beer_ids)]
 df_ba_ratings_filtered_beers_merged_users = df_ba_ratings_filtered_md_beers.merge(df_ba_users, left_on="user_id", right_on="user_id")
 print(df_ba_ratings_filtered_beers_merged_users.isnull().sum())

In [ ]:
# #### RateBeer Ratings
# ##### Load RB User Data

df_rb_users = pd.read_csv(path_rb + 'users.csv')
df_rb_users = df_rb_users.convert_dtypes(infer_objects=True) 
df_rb_users.head()

print(df_rb_users.isna().sum())

# remove lines with nan
df_rb_users = df_rb_users.dropna()

# split US states
df_rb_users['country'] = df_rb_users['location'].apply(lambda x : x.split(',', 1)[0])

country_list = df_rb_users['country'].unique()
country_iso = {x: find_iso(x) for x in country_list} #look up table
df_rb_users['country_code'] = df_rb_users['country'].apply(lambda x: country_iso[x])
print("No country code assignment to the following countries: ", df_rb_users[df_rb_users["country_code"].isnull()]["country"].unique())

# remove missing country codes
df_rb_users = df_rb_users.dropna()
print(df_rb_users.isna().sum())
print(df_rb_users.info())


# ##### Load BA Ratings
# TODO Explain difference between ratings and reviews
# TODO: We can leave this commented:
# Compute intersection
# df_ba_ratings_reviews = df_ba_ratings.merge(df_ba_reviews, how = "inner", on = ["beer_name", "beer_id", "brewery_name", "brewery_id", "style", "abv",  "user_name", "user_id", "appearance", "aroma", "palate", "taste", "overall", "rating"])
# df_ba_ratings_reviews.describe()
# df_ba_ratings_reviews.isna().sum()
# print("size of intersection in review/ratings BA", df_ba_ratings_reviews[~df_ba_ratings_reviews.isnull().any(axis=1)].count())
# print("size of reviews BA", df_ba_reviews[~df_ba_reviews.isnull().any(axis=1)].count())


# df_rb_reviews = pickle_load(path_rb + "reviews.txt",load_with_text=True)
# df_rb_reviews.set_index(df_rb_reviews["date"], inplace = True) # Set date as index for ratings
# df_rb_reviews = df_rb_reviews.convert_dtypes(infer_objects=True) # infer correct string types (does not filter nan values)
# df_rb_reviews[["overall","taste","palate","aroma","appearance","abv","user_id"]] = df_rb_reviews[["overall","taste","palate","aroma","appearance","abv","user_id"]].apply(lambda x : pd.to_numeric(x,errors="coerce")) # coerce to numeric
# df_rb_reviews["text"] = df_rb_reviews["text"].apply(lambda x: x if str.lower(x) != "nan" else "") # convert nan to empty string
# df_rb_reviews = df_rb_reviews.dropna() # drop nan values
# print(df_rb_reviews.info()) # print info
# print("Number of nan values",df_rb_reviews.isna().sum())
# print("Number of text that is 'nan':",df_rb_reviews[df_rb_reviews["text"].str.lower() == "nan"].shape[0])
# for column in ["beer_name","beer_id","brewery_name","brewery_name","style","user_name","user_id"]:
#         print(f"number of values in {column} that is 'nan'", df_rb_reviews[df_rb_reviews[column].str.lower() == "nan"].shape[0])




# ##### Merge BA Ratings and Users for beers from the matched Dataset
# Filter RB Ratings to only keep beers in matched beer dataset
# df_md_beer_ids = df_beers["beer_id_rb"].unique().astype(str)
# df_rb_beer_ids = df_rb_reviews["beer_id"].unique()
# df_rb_reviews_filtered_md_beers = df_rb_reviews[df_rb_reviews["beer_id"].isin(df_md_beer_ids)]
# df_rb_reviews_filtered_beers_merged_users = df_rb_reviews_filtered_md_beers.merge(df_rb_users, left_on="user_id", right_on="user_id")
# print(df_rb_reviews_filtered_beers_merged_users.isnull().sum())

pickle_filename = "df_rb_reviews_filtered_beers_merged_users.pickle"
# df_rb_reviews_filtered_beers_merged_users.to_pickle(f"Data/{pickle_filename}")
df_rb_reviews_filtered_beers_merged_users = pd.read_pickle(f"Data/{pickle_filename}")
print(df_rb_reviews_filtered_beers_merged_users.head())
print(df_rb_reviews_filtered_beers_merged_users.info())


# code

